# 計算股票的技術分析參考線

In [3]:
import os 
import sys
import pandas as pd

def calculate_sma(data, periods):
    '''
    計算均線
    輸入陣列類型的天數
    EX:[5,10,20]，就可以計算五日&十日&月均線的資料
    '''
    for period in periods:
        data[f'SMA{period}'] = data['Close'].rolling(window=period).mean()

def calculate_kdj(data, n=9, m1=3, m2=3):
    '''
    計算股票KDJ的數值
    '''
    # Calculate highest high and lowest low
    data['HighestHigh'] = data['High'].rolling(window=n).max()
    data['LowestLow'] = data['Low'].rolling(window=n).min()
    
    # Calculate RSV (Raw Stochastic Value)
    data['RSV'] = 100 * (data['Close'] - data['LowestLow']) / (data['HighestHigh'] - data['LowestLow'])
    
    # Calculate K value (Fast Stochastic)
    data['KDJ_K'] = data['RSV'].rolling(window=m1).mean()
    
    # Calculate D value (Slow Stochastic)
    data['KDJ_D'] = data['KDJ_K'].rolling(window=m2).mean()
    
    # Calculate J value
    data['KDJ_J'] = 3 * data['KDJ_K'] - 2 * data['KDJ_D']
    


def calculate_macd(data, short_period=12, long_period=26, signal_period=9):
    '''
    計算股票MACD 的數值
    '''
    # 計算短期（12日）和長期（26日）移動平均線
    short_ema = data['Close'].ewm(span=short_period, adjust=False).mean()
    long_ema = data['Close'].ewm(span=long_period, adjust=False).mean()

    # 計算DIF（快速線）和DEA（慢速線）
    data['dif'] = short_ema - long_ema
    data['dea'] = data['dif'].ewm(span=signal_period, adjust=False).mean()

    # 計算MACD柱狀圖（MACD Histogram）
    data['macd_hist'] = data['dif']  - data['dea']



def calculate_labels(data):
    '''
    紀錄標籤資料
    '''
    data['Up_3days'] = (data['Close'].shift(-5) >= data['Close'] * 1.05).astype(int)
    data['Down_3days'] = (data['Close'].shift(-5) <= data['Close']* 0.95).astype(int)
    data['Stable_3days'] = data['Close'].shift(-5).between(data['Close'] * 0.97, data['Close'] * 1.03).astype(int)
    data['Other'] = (~(data['Up_3days'] | data['Down_3days'] | data['Stable_3days'])).astype(int) & 1
    data['Volume_5avg'] = data['Volume'].rolling(window=5).mean()
    data['Volume_flag'] = (data['Volume_5avg'] > 5000 * 1000).astype(int)



# 測試單一股票的計算分析
# 讀取股票數據
file_path = 'F:/python/pystock/data/original'
data = pd.read_csv(f'{file_path}/0050.TW.csv',parse_dates=['Date'],index_col=0).dropna()
data['Date'] = pd.to_datetime(data.index)
data = data.set_index('Date').sort_index()

# 計算均線
calculate_sma(data, [5, 10, 20, 60])

# 計算KDJ指標
calculate_kdj(data)

# 計算MACD指標
calculate_macd(data)

# 計算標籤
calculate_labels(data)

data


,Open,High,Low,Close,Adj Close,Volume,SMA5,SMA10,SMA20,SMA60,...,KDJ_J,dif,dea,macd_hist,Up_3days,Down_3days,Stable_3days,Other,Volume_5avg,Volume_flag
Date,,,,,,,,,,,,,,,,,,,,,
2009-01-02,32.869999,32.869999,32.869999,32.869999,32.869999,0,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.000000,0.000000,0,0,1,0,NaN,0
2009-01-05,35.169998,36.590000,30.570000,36.590000,36.590000,26338,NaN,NaN,NaN,NaN,...,NaN,0.296752,0.059350,0.237402,0,1,0,0,NaN,0
2009-01-06,36.590000,36.590000,33.930000,36.570000,36.570000,24377,NaN,NaN,NaN,NaN,...,NaN,0.524273,0.152335,0.371938,0,1,0,0,NaN,0
2009-01-07,34.700001,34.869999,32.209999,32.209999,32.209999,16672,NaN,NaN,NaN,NaN,...,NaN,0.348749,0.191618,0.157131,1,0,0,0,NaN,0
2009-01-08,33.599998,33.810001,30.160000,30.160000,30.160000,31357,33.680000,NaN,NaN,NaN,...,NaN,0.043724,0.162039,-0.118315,0,0,1,0,19748.8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-06-15,131.550003,132.000000,131.500000,131.899994,131.899994,11041119,130.350000,128.650000,126.445000,121.670000,...,98.729530,2.748705,2.290668,0.458037,0,0,0,1,12085910.8,1
2023-06-16,131.800003,131.800003,131.100006,131.550003,131.550003,7202216,131.100000,129.130000,126.897500,121.894167,...,94.665197,2.789061,2.390347,0.398715,0,0,0,1,12415963.4,1
2023-06-19,130.949997,131.500000,130.550003,131.250000,131.250000,5978996,131.550000,129.620000,127.345000,122.104167,...,88.966149,2.764963,2.465270,0.299694,0,0,0,1,10867505.2,1


# 將所有的股票資料加入計算分析的數值

In [18]:
import random 
import csv

def saveLab(filename, listvalue ):
    '''
    list 寫到CSV檔案
    '''
    random.shuffle(listvalue)
    data_size = len(listvalue)
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)

        # Write the list as a single row in the CSV file
        writer.writerow(listvalue)

    print(f"List has been written to {filename} and size:{data_size}")



file_path = 'F:/python/pystock/data/original'
file_list = os.listdir(file_path)
output_path = 'F:/python/pystock/data/clean'
lst_labels_train= []
lst_labels_validation = []
lst_labels_test= []

for x in file_list:
    data = pd.read_csv(f'{file_path}/{x}')
    calculate_sma(data, [5, 10, 20, 60])

    # 計算KDJ指標
    calculate_kdj(data)

    # 計算MACD指標
    calculate_macd(data)
    data= data[60:]
    # 計算標籤
    calculate_labels(data)
    data.to_csv(f'{output_path}/{x}')
    
    filtered_data = data[data['Volume_flag'] == 1]
    filtered_data.dropna() # 必須排除無效的 標籤數值
    for d in filtered_data.Date:
        if d > '2010-01-01':
            if d < '2019-01-01':
                lst_labels_train.append(f'{x}_{d}')
            elif d > '2021-01-01':
                lst_labels_test.append(f'{x}_{d}')
            else:
                lst_labels_validation.append(f'{x}_{d}')

# 將標籤檔案寫道資料中    
output_label_path = 'F:/python/pystock/data/'
saveLab(f'{output_label_path}lst_labels_train.csv', lst_labels_train )
saveLab(f'{output_label_path}lst_labels_validation.csv',  lst_labels_validation )
saveLab(f'{output_label_path}lst_labels_test.csv',  lst_labels_test )    


List has been written to F:/python/pystock/data/lst_labels_train.csv and size:256447
List has been written to F:/python/pystock/data/lst_labels_validation.csv and size:72884
List has been written to F:/python/pystock/data/lst_labels_test.csv and size:118151


In [19]:
# 檢查一下當前的資料
# file_path = 'F:/python/pystock/data/clean'
# data = pd.read_csv(f'{file_path}/0050.TW.csv',parse_dates=['Date'],index_col=0).dropna()
# data

# 將資料寫入 mongodb

In [21]:
from pymongo import MongoClient

# 建立MongoDB連線
client = MongoClient('localhost', 27017)

# 建立或選擇MongoDB資料庫
db = client['cleanData']
# 股票資料寫入
input_mongodb_path = 'F:/python/pystock/data/clean'


file_list = os.listdir(input_mongodb_path)
for x in file_list:
    # 建立或選擇MongoDB的collection
    collection = db[x]
    # 讀取CSV檔案
    data = pd.read_csv(f'{input_mongodb_path}/0050.TW.csv',parse_dates=['Date'],index_col=0).dropna()
    # 把資料轉成dict格式並儲存到MongoDB
    collection.insert_many(data.to_dict('records'))
    
# 標籤資料寫入
def save_label(path ='F:/python/pystock/data/' , collection_name='nan') :
    
    with open(f'{root_path}{collection_name}.csv', 'r') as file:
        data_str = file.read()

    # 將字串以逗號分割為列表
    data_list = data_str.split(',')
    collection = db[collection_name]
    # 將列表中的每一個元素轉為字典並插入到MongoDB
    for value in data_list:
        collection.insert_one({"label": value})
        
labels = ['lst_labels_train','lst_labels_validation','lst_labels_test' ]
for label in labels:
    save_label(collection_name = label)

## 檢查 放入mongodb的股票資料

In [23]:

# 打印 股票
test_stock_id ='0050.TW.csv'
df = pd.DataFrame(list(db[test_stock_id].find().sort('Date', -1)))
df

,_id,Date,Open,High,Low,Close,Adj Close,Volume,SMA5,SMA10,...,KDJ_J,dif,dea,macd_hist,Up_3days,Down_3days,Stable_3days,Other,Volume_5avg,Volume_flag
0,64b212c64cc0e0ec9905d1b6,2023-06-21,130.550003,131.050003,130.250000,130.550003,130.550003,26565515,131.160001,130.250001,...,59.913835,2.545206,2.512211,0.032995,0,0,0,1,11463363.0,1
1,64b212c64cc0e0ec9905d1b5,2023-06-20,131.100006,131.250000,130.350006,130.550003,130.550003,6528969,131.350000,130.005001,...,71.483299,2.658734,2.503963,0.154771,0,0,0,1,7485658.6,1
2,64b212c64cc0e0ec9905d1b4,2023-06-19,130.949997,131.500000,130.550003,131.250000,131.250000,5978996,131.550000,129.620000,...,88.966149,2.764963,2.465270,0.299694,0,0,0,1,10867505.2,1
3,64b212c64cc0e0ec9905d1b3,2023-06-16,131.800003,131.800003,131.100006,131.550003,131.550003,7202216,131.100000,129.130000,...,94.665197,2.789061,2.390347,0.398715,0,0,0,1,12415963.4,1
4,64b212c64cc0e0ec9905d1b2,2023-06-15,131.550003,132.000000,131.500000,131.899994,131.899994,11041119,130.350000,128.650000,...,98.729530,2.748705,2.290668,0.458037,0,0,0,1,12085910.8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3483,64b212c64cc0e0ec9905c41b,2009-04-20,37.090000,40.500000,37.090000,40.169998,40.169998,20721,40.400000,39.796000,...,38.070740,1.381782,1.352493,0.029289,0,0,1,0,25517.6,0
3484,64b212c64cc0e0ec9905c41a,2009-04-17,44.080002,44.080002,38.320000,39.880001,39.880001,21658,40.402000,39.658000,...,44.174104,1.428910,1.345170,0.083740,0,0,1,0,26332.6,0
3485,64b212c64cc0e0ec9905c419,2009-04-16,43.060001,43.060001,40.900002,41.200001,41.200001,35606,40.420000,39.530000,...,63.394570,1.497806,1.324235,0.173571,0,0,1,0,33777.2,0
3486,64b212c64cc0e0ec9905c418,2009-04-15,40.500000,40.500000,39.820000,40.250000,40.250000,20582,40.036000,39.268000,...,61.005638,1.429556,1.280843,0.148713,0,0,1,0,31454.4,0


In [34]:
table_name = "lst_labels_train"
labels =pd.DataFrame(list( db[table_name].find({}, {'label': 1})))
labels

,_id,label
0,64b2179f4cc0e0ec99631e5a,2330.TW.csv_2013-12-23
1,64b2179f4cc0e0ec99631e5b,2408.TW.csv_2018-08-24
2,64b2179f4cc0e0ec99631e5c,6435.TWO.csv_2017-10-31
3,64b2179f4cc0e0ec99631e5d,2890.TW.csv_2016-06-29
4,64b2179f4cc0e0ec99631e5e,2889.TW.csv_2010-12-31
...,...,...
256442,64b217cd4cc0e0ec99670814,2881.TW.csv_2014-05-06
256443,64b217cd4cc0e0ec99670815,2353.TW.csv_2017-10-27
256444,64b217cd4cc0e0ec99670816,2109.TW.csv_2010-04-08
256445,64b217cd4cc0e0ec99670817,2352.TW.csv_2010-04-21
